<a href="https://colab.research.google.com/github/Re14m/isk/blob/master/2022_0223_nlp_getting_started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#GoogleDriveのKaggleフォルダの中に、[コンペ名]のフォルダを作る

In [ ]:
#Kaggleのコンペデータのzipを[コンペ名フォルダ]に入れる

In [ ]:
#コンペのzipをコピー
!cp "/content/drive/MyDrive/Kaggle/nlp-getting-started/nlp-getting-started.zip" .

In [ ]:
#zipを展開
!unzip -o /content/drive/MyDrive/Kaggle/nlp-getting-started/nlp-getting-started.zip -d .

Archive:  /content/drive/MyDrive/Kaggle/nlp-getting-started/nlp-getting-started.zip
  inflating: ./sample_submission.csv  
  inflating: ./test.csv              
  inflating: ./train.csv             


In [ ]:
#ライブラリのインポート
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 

from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import  Counter
plt.style.use('ggplot')
stop=set(stopwords.words('english'))
import re
from nltk.tokenize import word_tokenize
import gensim
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
tweet= pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [ ]:
df = pd.concat([tweet,test])
df.shape

(10876, 5)

In [ ]:
example = "New competition launched :https://www.kaggle.com/c/nlp-getting-started"

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

remove_URL(example)
df['text'] = df['text'].apply(lambda x : remove_URL(x))

In [ ]:
example = """<div>
<h1>Real or Fake</h1>
<p>Kaggle </p>
<a href="https://www.kaggle.com/c/nlp-getting-started">getting started</a>
</div>"""

def remove_html(text):
    html = re.compile(r'<.*?>')
    return html.sub(r'',text)
print(remove_html(example))

df['text'] = df['text'].apply(lambda x : remove_html(x))


Real or Fake
Kaggle 
getting started



In [ ]:
# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

remove_emoji("Omg another Earthquake 😔😔")

df['text'] = df['text'].apply(lambda x: remove_emoji(x))

In [ ]:
def remove_punct(text):
    table = str.maketrans('','', string.punctuation)
    return text.translate(table)

example = "I am a #king"
print(remove_punct(example))

df['text'] = df['text'].apply(lambda x : remove_punct(x))

I am a king


In [ ]:
!pip install pyspellchecker

     |████████████████████████████████| 2.7 MB 5.7 MB/s 


In [ ]:
from spellchecker import SpellChecker

spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)
        
text = "corect me plese"
correct_spellings(text)

'correct me plese'

In [ ]:
def create_corpus(df):
    corpus=[]
    for tweet in tqdm(df['text']):
        words=[word.lower() for word in word_tokenize(tweet) if((word.isalpha()==1) & (word not in stop))]
        corpus.append(words)
    return corpus

corpus = create_corpus(df)

100%|██████████| 10876/10876 [00:04<00:00, 2292.62it/s]


In [ ]:
#コンペのフォルダに「https://www.kaggle.com/re1am0/notebook0fffe9debf/data?select=glove.6B.100d.txt」をDL

In [ ]:
#glove.6B.100d.txt.zipをコピー
!cp "/content/drive/MyDrive/Kaggle/nlp-getting-started/glove.6B.100d.txt.zip" .

In [ ]:
#glove.6B.100d.txt.zipを展開
!unzip -o "/content/drive/MyDrive/Kaggle/nlp-getting-started/glove.6B.100d.txt.zip" -d.

Archive:  /content/drive/MyDrive/Kaggle/nlp-getting-started/glove.6B.100d.txt.zip
  inflating: ./glove.6B.100d.txt     


In [ ]:
#glove.6B.100d.txtを使う
embedding_dict={}
with open('glove.6B.100d.txt','r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:],'float32')
        embedding_dict[word]  =vectors
f.close()

MAX_LEN = 50
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(corpus)
sequences = tokenizer_obj.texts_to_sequences(corpus)

tweet_pad = pad_sequences(sequences, maxlen=MAX_LEN, truncating='post', padding='post')

word_index = tokenizer_obj.word_index
print('Number of unique words:',len(word_index))

Number of unique words: 20342


In [ ]:
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, 100))

for word,i in tqdm(word_index.items()):
    if i > num_words:
        continue
    
    emb_vec = embedding_dict.get(word)
    if emb_vec is not None:
        embedding_matrix[i] = emb_vec

100%|██████████| 20342/20342 [00:00<00:00, 307391.59it/s]


In [ ]:
model = Sequential()

embedding=Embedding(num_words, 100, embeddings_initializer=Constant(embedding_matrix),
                   input_length=MAX_LEN, trainable=False)

model.add(embedding)
model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

optimzer = Adam(learning_rate=1e-5)

model.compile(loss='binary_crossentropy',optimizer=optimzer,metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 100)           2034300   
                                                                 
 spatial_dropout1d (SpatialD  (None, 50, 100)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 64)                42240     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 2,076,605
Trainable params: 42,305
Non-trainable params: 2,034,300
_________________________________________________________________


In [ ]:
train=tweet_pad[:tweet.shape[0]]
test=tweet_pad[tweet.shape[0]:]

X_train,X_test,y_train,y_test = train_test_split(train,tweet['target'].values,test_size=0.15)
print('Shape of train',X_train.shape)
print("Shape of Validation ",X_test.shape)

Shape of train (6471, 50)
Shape of Validation  (1142, 50)


In [ ]:
history=model.fit(X_train,y_train,batch_size=4,epochs=15,validation_data=(X_test,y_test),verbose=2)

Epoch 1/15
1618/1618 - 529s - loss: 0.6912 - accuracy: 0.5653 - val_loss: 0.6869 - val_accuracy: 0.5884 - 529s/epoch - 327ms/step
Epoch 2/15
1618/1618 - 508s - loss: 0.6355 - accuracy: 0.6390 - val_loss: 0.5545 - val_accuracy: 0.7925 - 508s/epoch - 314ms/step
Epoch 3/15
1618/1618 - 499s - loss: 0.5585 - accuracy: 0.7534 - val_loss: 0.5132 - val_accuracy: 0.7776 - 499s/epoch - 309ms/step
Epoch 4/15
1618/1618 - 497s - loss: 0.5343 - accuracy: 0.7586 - val_loss: 0.4935 - val_accuracy: 0.7916 - 497s/epoch - 307ms/step
Epoch 5/15
1618/1618 - 494s - loss: 0.5274 - accuracy: 0.7534 - val_loss: 0.4777 - val_accuracy: 0.8004 - 494s/epoch - 306ms/step
Epoch 6/15
1618/1618 - 498s - loss: 0.5104 - accuracy: 0.7648 - val_loss: 0.4667 - val_accuracy: 0.8039 - 498s/epoch - 308ms/step
Epoch 7/15
1618/1618 - 502s - loss: 0.5006 - accuracy: 0.7742 - val_loss: 0.4603 - val_accuracy: 0.8021 - 502s/epoch - 310ms/step
Epoch 8/15
1618/1618 - 493s - loss: 0.5008 - accuracy: 0.7725 - val_loss: 0.4584 - val_acc

In [ ]:
sample_sub=pd.read_csv('sample_submission.csv')

y_pre = model.predict(test)
y_pre = np.round(y_pre).astype(int).reshape(3263)
sub = pd.DataFrame({'id':sample_sub['id'].values.tolist(), 'target':y_pre})
sub.to_csv('submission.csv',index=False)